In [ ]:
!pip install tf-agents[reverb]
!pip install tf-keras

In [ ]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

In [ ]:
import random
'''
0 1 2
3 4 5
6 7 8
'''
class TicTacToeEnv(py_environment.PyEnvironment):

    def __init__(self):

        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=8, name='action') # user can choose to put on one of 9 places on board

        self._observation_spec = array_spec.BoundedArraySpec(shape=(1,9), dtype=np.int32, minimum=0, maximum=2, name='observation') # 3 by 3 grid

        self._episode_ended = False
        self._state = [0,0,0,0,0,0,0,0,0]
        self.current_player = 1 # human player
        self._outcome = 0 # 0 for no winner, 1 for player 1 wins, 2 for player 2, and 3 for draw
        self._moves = 0

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._episode_ended = False
        self.current_player = 1
        self._state = [0,0,0,0,0,0,0,0,0]
        self._moves = 0
        return ts.restart(np.array([self._state], dtype=np.int32))



    def _check_game_over(self):
        for j in range(3):
          if self._state[j*3] == self._state[j*3 + 1] == self._state[j*3 + 2] != 0:
            self._outcome = self._state[j*3]  # Corrected assignment
            return True  # Row win

        for i in range(3):
          if self._state[i] == self._state[3+i] == self._state[6+i] != 0:
            self._outcome = self._state[i]  # Corrected assignment
            return True  # Column win

        if self._state[0] == self._state[4] == self._state[8] != 0:
            self._outcome = self._state[0]  # Corrected assignment
            return True  # Diagonal win
        if self._state[2] == self._state[4] == self._state[6] != 0:
            self._outcome = self._state[2]  # Corrected assignment
            return True  # Anti-diagonal win

        if 0 in self._state:
            return False  # Game is still ongoing

        self._outcome = 3
        return True  # Draw (no winner)


    def _change_board(self, action, player):
        if action < 3:
            self._state[action][0] = player
        elif action < 6:
            self._state[action-3][1] = player
        elif action < 9:
            self._state[action-6][2] = player

    def _check_block_opponent(self):
        opponent = 3 - self.current_player  # Determine the opponent's mark
        # Iterate through all possible winning combinations
        for combination in [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]:
            # Check if the opponent has two marks in a winning combination and the third position is empty
            if (self._state[combination[0]] == opponent and
                self._state[combination[1]] == opponent and
                self._state[combination[2]] == 0):
                return True  # Blocking move detected
            elif (self._state[combination[0]] == opponent and
                  self._state[combination[2]] == opponent and
                  self._state[combination[1]] == 0):
                return True  # Blocking move detected
            elif (self._state[combination[1]] == opponent and
                  self._state[combination[2]] == opponent and
                  self._state[combination[0]] == 0):
                return True  # Blocking move detected
        return False  # No blocking move detected


    def _step(self, action):
        if self._episode_ended:
            return self.reset()

        reward = 0
        if self._state[action] == 0:
            self._state[action] = self.current_player
        else:
            reward = -1  # Invalid move, penalize the agent

        # Check if the agent's move blocks the opponent from winning
        if self._check_block_opponent():
            reward += 1  # Add a positive reward for blocking the opponent's winning move

        self._episode_ended = self._check_game_over()

        self._moves += 1
        if self._moves == 9:
            self._episode_ended = True
            if not self._outcome:
                reward = 0  # Draw
        self.current_player = 3 - self.current_player  # Toggle between 1 and 2

        if self._episode_ended:
            if self._outcome == 3:
                reward = 0
            elif self._outcome == 1:
                reward = 5
            elif self._outcome == 2:
                reward = -5

            return ts.termination(
                np.array([self._state], dtype=np.int32), reward)
        else:
            return ts.transition(
                np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)


    # Function to convert state to index
    def state_to_index(self, state):
        return int(''.join(map(str, state)), 3)

    def choose_action(self, state, epsilon, agent):
        if np.random.rand() < epsilon:
            return np.random.randint(9)
        elif agent == 1:
            return np.argmax(Q_table_1[self.state_to_index(state)])
        else:
            return np.argmax(Q_table_1[self.state_to_index(state)])

    def opponent_strategy(self, state):
        # Check if opponent can win in the next move
        for i in range(9):
            if state[i] == 0:
                state[i] = 2
                if self._check_game_over():
                    state[i] = 0
                    return i
                state[i] = 0

        # Check if player can win in the next move and block it
        for i in range(9):
            if state[i] == 0:
                state[i] = 1
                if self._check_game_over():
                    state[i] = 0
                    return i
                state[i] = 0

        # Choose a random empty position
        empty_positions = [i for i in range(9) if state[i] == 0]
        if empty_positions:
          return np.random.choice(empty_positions)
        else:
          return 0

    def train(self, num_episodes, epsilon=0.1, alpha=0.1, gamma=0.9):
        Q_table_1 = np.zeros((3**9, 9))  # Q-table size: number of possible states x number of actions

        # Function to convert state to index
        def state_to_index(state):
            return int(''.join(map(str, state)), 3)

        # Function to choose action based on epsilon-greedy policy
        def choose_action(state, epsilon):
            if np.random.rand() < epsilon:
                return np.random.randint(9)
            else:
                return np.argmax(Q_table_1[state_to_index(state)])

        # Training loop
        for episode in range(num_episodes):
            time_step = self.reset()
            state = time_step.observation[0]
            done = False

            total_reward = 0

            while not done:
                # Agent 1's turn
                action = choose_action(state, epsilon)
                time_step = self.step(action)
                next_state = time_step.observation[0]
                reward = time_step.reward
                done = time_step.is_last()

                # Update Q-value for Agent 1
                next_state_index = state_to_index(next_state)
                if not done:
                    Q_table_1[state_to_index(state), action] += alpha * (reward + gamma * np.max(Q_table_1[next_state_index]) - Q_table_1[state_to_index(state), action])
                else:
                    Q_table_1[state_to_index(state), action] += alpha * (reward - Q_table_1[state_to_index(state), action])

                total_reward += reward

                if done:
                    break

                # Player 2's turn (random move)
                action = self.opponent_strategy(state)  # Randomly choose an action
                time_step = self.step(action)
                next_state = time_step.observation[0]
                reward = time_step.reward

                state = next_state

            # Print progress
            if episode % 100 == 0:
                print("Episode {}: Total Reward for Agent 1 = {}".format(episode, total_reward))

        return Q_table_1


    def play_against_agent(self, Q_table, first_turn):
        time_step = self.reset()
        current_state = time_step.observation[0]
        done = False

        if first_turn == 2:
          action = np.argmax(Q_table[self.state_to_index(current_state)])
        while not done:
            print("Current Board:")
            self.print_board(current_state)

            if self.current_player == 1:
                action = int(input("Your move (0-8): "))
                if action < 0 or action > 8:
                    print("Invalid move. Please choose a number between 0 and 8.")
                    continue
            else:
                action = np.argmax(Q_table[self.state_to_index(current_state)])

            time_step = self.step(action)
            current_state = time_step.observation[0]
            done = time_step.is_last()

        print("Game Over!")
        print("Result: ", end="")
        if self._outcome == 1:
            print("You win!")
        elif self._outcome == 2:
            print("Agent 1 wins!")
        else:
            print("It's a draw!")

    def print_board(self, state):
        symbols = ['_', 'X', 'O']
        for i in range(3):
            print(" ".join(symbols[state[j + i*3]] for j in range(3)))

In [ ]:
environment = TicTacToeEnv()
utils.validate_py_environment(environment, episodes=5)

In [ ]:
env = TicTacToeEnv()
Q_table_1 = env.train(num_episodes=5000)

In [ ]:
def save_q_table(Q_table, filename):
    np.save(filename, Q_table)

# Example usage:
# Assuming Q_table is the trained Q-table and filename is the path to save the file
save_q_table(Q_table_1, 'q_table.npy')

In [ ]:

env.play_against_agent(Q_table_1, 2)